In [1]:
import requests
from bs4 import BeautifulSoup
page=requests.get('https://books.toscrape.com/index.html')
soup = BeautifulSoup(page.text,'html.parser')

In [2]:
import sqlite3
con=sqlite3.connect('Books1.db')
curs=con.cursor()
curs.execute('create table data (Rates integer not null, Price integer not null, Title text not null)')
con.commit()
con.close()

In [5]:
con=sqlite3.connect('Books1.db')
curs=con.cursor()
rates_dic={'Zero':0,'One':1,'Two':2,'Three':3,'Four':4,'Five':5}
for product in soup.find_all('article',attrs={'class':'product_pod'}):
    rates=rates_dic[product.find('p').get('class')[1]]
    price=float(product.find('div',attrs={'class':'product_price'}).find('p').get_text()[2:])
    title=product.find('h3').find('a').get('title')
    curs.execute('insert into data (Rates,Price,Title) values(?,?,?)',[rates,price,title])
    con.commit()
        
while soup.find('li',attrs={'class':'next'}):
    domain='https://books.toscrape.com/catalogue/'
    Next=domain+soup.find('li',attrs={'class':'next'}).find('a').get('href')
    Next=Next.split('/')
    if Next[4]=='catalogue':
        del Next[4]
        Next=' '.join(Next)
        Next=Next.replace(' ','/')
    else:
        Next=domain+soup.find('li',attrs={'class':'next'}).find('a').get('href')  

    next_page=requests.get(Next)
    soup = BeautifulSoup(next_page.text,'html.parser')
    for product in soup.find_all('article',attrs={'class':'product_pod'}):
        rates=rates_dic[product.find('p').get('class')[1]]
        price=float(product.find('div',attrs={'class':'product_price'}).find('p').get_text()[2:])
        title=product.find('h3').find('a').get('title')
        curs.execute('insert into data (Rates,Price,Title) values(?,?,?)',[rates,price,title])
        con.commit()
con.close()

In [6]:
import pandas as pd
con=sqlite3.connect('Books1.db')
df=pd.read_sql('select * from data',con)
con.close()
df

,Rates,Price,Title
0,3,51.77,A Light in the Attic
1,1,53.74,Tipping the Velvet
2,1,50.10,Soumission
3,4,47.82,Sharp Objects
4,5,54.23,Sapiens: A Brief History of Humankind
...,...,...,...
1015,1,55.53,Alice in Wonderland (Alice's Adventures in Won...
1016,4,57.06,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)"
1017,5,16.97,A Spy's Devotion (The Regency Spies of London #1)
1018,1,53.98,1st to Die (Women's Murder Club #1)


In [7]:
con=sqlite3.connect('Books1.db')
con.execute('select * from data where Rates>= ? and Title like ?',[3,'%MR%']).fetchall()

[(5, 30.25, 'Mrs. Houdini'),
 (3, 24.48, 'My Mrs. Brown'),
 (5, 11.21, 'Dear Mr. Knightley')]

In [8]:
con=sqlite3.connect('Books1.db')
con.execute('select * from data where rates>= ? or price > ?',[3,20]).fetchall()

[(3, 51.77, 'A Light in the Attic'),
 (1, 53.74, 'Tipping the Velvet'),
 (1, 50.1, 'Soumission'),
 (4, 47.82, 'Sharp Objects'),
 (5, 54.23, 'Sapiens: A Brief History of Humankind'),
 (1, 22.65, 'The Requiem Red'),
 (4, 33.34, 'The Dirty Little Secrets of Getting Your Dream Job'),
 (3,
  17.93,
  'The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull'),
 (4,
  22.6,
  'The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics'),
 (1, 52.15, 'The Black Maria'),
 (4, 20.66, "Shakespeare's Sonnets"),
 (5, 17.46, 'Set Me Free'),
 (5, 52.29, "Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)"),
 (5, 35.02, 'Rip it Up and Start Again'),
 (3,
  57.25,
  'Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991'),
 (1, 23.88, 'Olio'),
 (1, 37.59, 'Mesaerion: The Best Science Fiction Stories 1800-1849'),
 (2, 51.33, 'Libertarianism for Beginners'),
 (2, 45.17, "It's Only the Himalayas"),
 (2, 3

In [9]:
con=sqlite3.connect('Books1.db')
con.execute('select*from Data where rates != ?',[3]).fetchall()

[(1, 53.74, 'Tipping the Velvet'),
 (1, 50.1, 'Soumission'),
 (4, 47.82, 'Sharp Objects'),
 (5, 54.23, 'Sapiens: A Brief History of Humankind'),
 (1, 22.65, 'The Requiem Red'),
 (4, 33.34, 'The Dirty Little Secrets of Getting Your Dream Job'),
 (4,
  22.6,
  'The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics'),
 (1, 52.15, 'The Black Maria'),
 (2, 13.99, 'Starving Hearts (Triangular Trade Trilogy, #1)'),
 (4, 20.66, "Shakespeare's Sonnets"),
 (5, 17.46, 'Set Me Free'),
 (5, 52.29, "Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)"),
 (5, 35.02, 'Rip it Up and Start Again'),
 (1, 23.88, 'Olio'),
 (1, 37.59, 'Mesaerion: The Best Science Fiction Stories 1800-1849'),
 (2, 51.33, 'Libertarianism for Beginners'),
 (2, 45.17, "It's Only the Himalayas"),
 (1, 12.84, 'In Her Wake'),
 (2, 37.32, 'How Music Works'),
 (5, 25.27, 'Chase Me (Paris Nights #2)'),
 (5, 34.53, 'Black Dust'),
 (5, 40.3, 'Worlds Elsewhere: Journeys Around Shakespeareâ\x8

In [10]:
con=sqlite3.connect('Books1.db')
con.execute('select*from Data where price between ? and ? and rates in (?,?,?)',[10,40,3,4,1]).fetchall()

[(1, 22.65, 'The Requiem Red'),
 (4, 33.34, 'The Dirty Little Secrets of Getting Your Dream Job'),
 (3,
  17.93,
  'The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull'),
 (4,
  22.6,
  'The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics'),
 (4, 20.66, "Shakespeare's Sonnets"),
 (1, 23.88, 'Olio'),
 (1, 37.59, 'Mesaerion: The Best Science Fiction Stories 1800-1849'),
 (1, 12.84, 'In Her Wake'),
 (3,
  30.52,
  'Foolproof Preserving: A Guide to Small Batch Jams, Jellies, Pickles, Condiments, and More: A Foolproof Guide to Making Small Batch Jams, Jellies, Pickles, Condiments, and More'),
 (3,
  22.5,
  "America's Cradle of Quarterbacks: Western Pennsylvania's Football Factory from Johnny Unitas to Joe Montana"),
 (3,
  31.05,
  'The Five Love Languages: How to Express Heartfelt Commitment to Your Mate'),
 (1, 36.89, 'The Bear and the Piano'),
 (3, 33.29, 'Penny Maybe'),
 (1, 19.63, 'In a Dark, Dark Wood'),
 

In [11]:
con=sqlite3.connect('Books1.db')
con.execute("select max(price) as mp , title from Data ORDER BY Price DESC LIMIT 5 ").fetchall()

[(59.99, 'The Perfect Play (Play by Play #1)')]

In [12]:
con=sqlite3.connect('Books1.db')
con.execute('select * from Data order by rates , price asc limit 30 ').fetchall()

[(1, 10.4, 'The Lucifer Effect: Understanding How Good People Turn Evil'),
 (1, 10.62, 'Counting Thyme'),
 (1, 10.65, 'The Project'),
 (1, 10.69, 'Tastes Like Fear (DI Marnie Rome #3)'),
 (1,
  10.76,
  'Miss Peregrineâ\x80\x99s Home for Peculiar Children (Miss Peregrineâ\x80\x99s Peculiar Children #1)'),
 (1,
  10.92,
  "The Restaurant at the End of the Universe (Hitchhiker's Guide to the Galaxy #2)"),
 (1,
  10.97,
  'The Long Shadow of Small Ghosts: Murder and Memory in an American City'),
 (1, 11.84, 'Hide Away (Eve Duncan #20)'),
 (1, 12.25, 'The Purest Hook (Second Circle Tattoos #3)'),
 (1, 12.3, 'The Giver (The Giver Quartet #1)'),
 (1, 12.84, 'In Her Wake'),
 (1, 12.87, 'Dark Lover (Black Dagger Brotherhood #1)'),
 (1, 13.51, 'Night (The Night Trilogy #1)'),
 (1, 13.92, 'That Darkness (Gardiner and Renner #1)'),
 (1, 14.16, 'Outliers: The Story of Success'),
 (1, 14.4, 'The End of the Jesus Era (An Investigation #1)'),
 (1, 14.64, 'Two Summers'),
 (1, 14.74, 'Harry Potter and 

In [14]:
con=sqlite3.connect('Books1.db')
con.execute("INSERT INTO Data (rates,price,title) VALUES (?,?,?)",[2,34.22,'handing'])
con.commit()
con.close()

In [15]:
con=sqlite3.connect('Books1.db')
con.execute("INSERT INTO Data (rates,price,title) VALUES (?,?,?)",[4,21.33,'cure world'])
con.commit()
con.close()

In [16]:
con=sqlite3.connect('Books1.db')
con.execute('select * from data where title = ? ',['handing']).fetchall()

[(2, 34.22, 'handing'), (2, 34.22, 'handing')]

In [17]:
con=sqlite3.connect('Books1.db')
con.execute('update Data set rates = ? where price<= ?',[3,20])
con.commit()
con.close()

In [18]:
con=sqlite3.connect('Books1.db')
con.execute('select title,rates from data where price<= ?',[20]).fetchall()

[('The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull',
  3),
 ('Starving Hearts (Triangular Trade Trilogy, #1)', 3),
 ('Set Me Free', 3),
 ('In Her Wake', 3),
 ('The Four Agreements: A Practical Guide to Personal Freedom', 3),
 ("Sophie's World", 3),
 ('Maude (1883-1993):She Grew Up with the country', 3),
 ('In a Dark, Dark Wood', 3),
 ('Untitled Collection: Sabbath Poems 2014', 3),
 ('Unicorn Tracks', 3),
 ('Tsubasa: WoRLD CHRoNiCLE 2 (Tsubasa WoRLD CHRoNiCLE #2)', 3),
 ('This One Summer', 3),
 ('Thirst', 3),
 ('The Torch Is Passed: A Harding Family Story', 3),
 ('The Life-Changing Magic of Tidying Up: The Japanese Art of Decluttering and Organizing',
  3),
 ('The Age of Genius: The Seventeenth Century and the Birth of the Modern Mind',
  3),
 ('Reskilling America: Learning to Labor in the Twenty-First Century', 3),
 ('Princess Jellyfish 2-in-1 Omnibus, Vol. 01 (Princess Jellyfish 2-in-1 Omnibus #1)',
  3),
 ('Princess Between Worlds (Wide-Awake P

In [19]:
con=sqlite3.connect('Books1.db')
curs=con.cursor()
curs.execute('delete from data where price > ? and rates <= ?',[50,2])
con.commit()
con.close()

In [20]:
con=sqlite3.connect('Books1.db')
con.execute('select count(title) from data where title like ? and price between ? and ? ',['%secret%',10,25]).fetchall()

[(5,)]

In [21]:
con=sqlite3.connect('Books1.db')
con.execute('select max(price),min(price) from data where rates = ?',[5]).fetchall()

[(59.92, 20.02)]

In [22]:
con=sqlite3.connect('Books1.db')
con.execute('select avg(price) from data where rates = ?',[5]).fetchall()

[(40.940443037974674,)]

In [23]:
con=sqlite3.connect('Books1.db')
con.execute('select sum(title) from data where rates = ? and price between ? and ?',[2,10,40]).fetchall()

[(33.0,)]

In [24]:
df.to_csv('Books1.csv',index = False)

In [25]:
df = pd.read_csv('Books1.csv')
df

,Rates,Price,Title
0,3,51.77,A Light in the Attic
1,1,53.74,Tipping the Velvet
2,1,50.10,Soumission
3,4,47.82,Sharp Objects
4,5,54.23,Sapiens: A Brief History of Humankind
...,...,...,...
1015,1,55.53,Alice in Wonderland (Alice's Adventures in Won...
1016,4,57.06,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)"
1017,5,16.97,A Spy's Devotion (The Regency Spies of London #1)
1018,1,53.98,1st to Die (Women's Murder Club #1)
